In [284]:
#Let's imagine these are the messages in our inbox. Our hypothesis is 
#that the percentage of spam in our inbox is greater than 12.5%.

#Alpha is 0.025

#Do we reject or not reject the null hypothesis? What's the p-value?

from scipy.stats import norm, t, ttest_1samp, ttest_ind, poisson
import numpy as np
from numpy import genfromtxt
from numpy import random
import pandas as pd

In [285]:
df = pd.read_csv('spam.csv', sep = ",",  encoding = 'latin-1')

In [286]:
outcomes = df['v1']

#binarize outcomes
bin_outcomes = []
for outcome in outcomes:
    if outcome == 'ham':
        bin_outcomes.append(0)
    else:
        bin_outcomes.append(1)

outcomes = bin_outcomes


In [287]:
mean_null_hypothesis = .125
#alt_hypothesis > .125
a = .025
#prop = np.sqrt(((0.125) * (1-.125)/300))
prop = np.sqrt(((mean_null_hypothesis) * (1-mean_null_hypothesis)/len(outcomes)))
x_bar = sum(outcomes)/len(outcomes)

In [288]:
rv = norm(mean_null_hypothesis,prop)
#spam_df = df[df['v1'] == 'spam']['v1']
1 - rv.cdf(x_bar)

0.020396461702911872

In [289]:
l = sum(outcomes)/28
rvp = poisson(l)
1 - rvp.cdf(29) #prob of at least 30 spam per day

0.2847229977401552

In [290]:
# Human comparison (Part 3)
import random
num = df.shape[0]
results = []
for i in range(10):
    index = np.random.randint(0, num)
    print(df.iloc[index][1])
    results.append(df.iloc[index][0]) 
 


If you hear a loud scream in about &lt;#&gt; minutes its cause my Gyno will be shoving things up me that don't belong :/
Sorry,in meeting I'll call later
Thanks honey. Have a great day.
Good evening Sir, hope you are having a nice day. I wanted to bring it to your notice that I have been late in paying rent for the past few months and have had to pay a $ &lt;#&gt;  charge. I felt it would be inconsiderate of me to nag about something you give at great cost to yourself and that's why i didnt speak up. I however am in a recession and wont be able to pay the charge this month hence my askin well ahead of month's end. Can you please help. Thank you for everything.
Am also doing in cbe only. But have to pay.
Freemsg: 1-month unlimited free calls! Activate SmartCall Txt: CALL to No: 68866. Subscriptn3gbp/wk unlimited calls Help: 08448714184 Stop?txt stop landlineonly
I shall book chez jules for half eight, if that's ok with you?
Em, its olowoyey@ usc.edu have a great time in argentina. Not s

In [291]:
#random.seed(9001)
msg_dict={}
for i in range(10):
    index=np.random.randint(1,5572)
    msg_dict[index] = tuple([df['v2'][index],df['v1'][index]])
msg_dict.keys()
for i in msg_dict.keys():
    print(i,msg_dict[i][0])

5520 No. I dont want to hear anything
4438 i want to grasp your pretty booty :)
1510 Pete,is this your phone still? Its Jenny from college and Leanne.what are you up to now?:)
1196 What he said is not the matter. My mind saying some other matter is there.
3837 Went to pay rent. So i had to go to the bank to authorise the payment.
1435 Dad went out oredi... 
101 I'm still looking for a car to buy. And have not gone 4the driving test yet.
4734 Oh k:)after that placement there ah?
2869 YOUR CHANCE TO BE ON A REALITY FANTASY SHOW call now = 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national = rate call.
4186 Dear got bus directly to calicut


In [292]:
for i in msg_dict.keys():
    print(i,msg_dict[i][0], msg_dict[i][1])

5520 No. I dont want to hear anything ham
4438 i want to grasp your pretty booty :) ham
1510 Pete,is this your phone still? Its Jenny from college and Leanne.what are you up to now?:) ham
1196 What he said is not the matter. My mind saying some other matter is there. ham
3837 Went to pay rent. So i had to go to the bank to authorise the payment. ham
1435 Dad went out oredi...  ham
101 I'm still looking for a car to buy. And have not gone 4the driving test yet. ham
4734 Oh k:)after that placement there ah? ham
2869 YOUR CHANCE TO BE ON A REALITY FANTASY SHOW call now = 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national = rate call. spam
4186 Dear got bus directly to calicut ham


In [293]:
# Part V
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

counts = count_vect.fit_transform(df['v2'])  
tfidfs = tfidf_transformer.fit_transform(counts)

X_train, X_test, y_train, y_test = train_test_split(tfidfs, df['v1'], test_size=0.33, random_state=42) #change

nb = MultinomialNB().fit(X_train, y_train)
predictions = nb.predict(X_test) 

In [294]:
raw = {'actual': y_test, 'pred' : predictions}
test_results_df = pd.DataFrame(data = raw)
test_results_df['actual']
correct_df = test_results_df[test_results_df['actual'] == test_results_df['pred']]

In [295]:
len(correct_df)/len(test_results_df) # percentage of correct predictions of total messages

0.953779227841218

In [296]:
y_test.value_counts()
1582/(252 + 1582)

0.8625954198473282

In [297]:
incorrect_df = acc_df[acc_df['actual'] != acc_df['pred']]

In [298]:
false_negatives = len(incorrect_df[incorrect_df['actual'] == 'spam']) # this is where we predicted spam when it was actually ham
false_positives = len(incorrect_df[incorrect_df['actual'] == 'ham'])  # this is where we predicted ham when it was actually spam
print('false negatives: {}  false positives {}'.format(false_negatives, false_positives))

false negatives: 85  false positives 0


In [299]:
# We run our 
new_df = test_results_df[test_results_df['pred'] == 'ham']
new_acc = len(new_df[new_df['actual'] == new_df['pred']])/len(new_df)
spam_prop = np.sqrt((1 - new_acc)*(1-(1 - new_acc))/len(new_df))

In [300]:
rv_new = norm(mean_null_hypothesis, spam_prop)

In [301]:
1 - rv.cdf(new_acc)

0.0